## **Using Mobile Phone**

In [1]:
import cv2
from urllib.request import urlopen
import numpy as np
from ultralytics import YOLO

# Define the path to the model weights
model_weights = 'runs/detect/train4/weights/best.pt'

# Create a YOLO object with the specified model weights and set the confidence threshold
yolo = YOLO(model_weights)

# Define the IP address and port number of your phone's camera
# Replace 'http://192.168.0.100:8080' with your phone's camera IP and port
phone_camera_url = 'http://192.168.0.104:8080/video'

# Open a handle to the phone's camera feed
cap = cv2.VideoCapture(phone_camera_url)

# Ensure the camera feed opened correctly
if not cap.isOpened():
    raise IOError("Cannot open phone's camera")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Perform detection
    results = yolo.track(source=0, show=True, tracker='bytetrack.yaml')

    # The correct way to access detections is results.xyxy[0] for the first image
    detections = results[0]  # This should now properly contain the detections

    # Check if there are any detections
    if len(detections) > 0:
        for detection in detections:
            x1, y1, x2, y2, conf, class_id = detection.cpu().numpy()
            label = yolo.names[int(class_id)]
            
            # Filter for specific classes (e.g., traffic lights)
            if label in ['red_pedestrian_light', 'green_pedestrian_light']:  # Adjust these labels as per your model's output
                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow('Real-time Traffic Light Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()


OSError: Cannot open phone's camera